<a href="https://colab.research.google.com/github/arjun251/Text2Json/blob/main/JsonWithObjCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Approach 1 - Using KeyBERT and get all keys**

In [ ]:
from keybert import KeyBERT

In [ ]:
!pip install keybert

     |████████████████████████████████| 79 kB 2.9 MB/s 
     |████████████████████████████████| 224 kB 11.3 MB/s 
     |████████████████████████████████| 51 kB 5.7 MB/s 
     |████████████████████████████████| 3.8 MB 38.3 MB/s 
     |████████████████████████████████| 1.2 MB 29.8 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 6.5 MB 40.7 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=db18fa2dd05b981d766fc7c890c8ce13333d665221d01f573387b29c51c626e6
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=857c1a9a101da8d9281d250ed30170c0c512d9a38b5b0aa6a5ba4e7ef342d786
  Stored in directory: /root/.cache/pip/whee

In [ ]:
model_keyBert = KeyBERT('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
input_text = "I see five cars one white and four black and two pedestrians"

In [ ]:
keywords = model_keyBert.extract_keywords(input_text,keyphrase_ngram_range=(1, 2), highlight = True, stop_words=None)

I see five cars white and four black two pedestrians

In [ ]:
keyword_items = list(map(lambda x:x[0], keywords))
print(keyword_items)

['five cars', 'cars', 'four black', 'cars one', 'two pedestrians']


['five cars', 'cars', 'four black', 'cars one', 'two pedestrians']

In [ ]:
def text2int(textnum, numwords={}):
  curstring = ""
  try:
    if not numwords:
        units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]

        tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

        scales = ["hundred", "thousand", "million", "billion", "trillion"]

        numwords["and"] = (1, 0)
        for idx, word in enumerate(units):  numwords[word] = (1, idx)
        for idx, word in enumerate(tens):       numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]

    textnum = textnum.replace('-', ' ')

    current = result = 0
    
    onnumber = False
    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if word not in numwords:
                if onnumber:
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
            else:
                scale, increment = numwords[word]

                current = current * scale + increment
                if scale > 100:
                    result += current
                    current = 0
                onnumber = True

    if onnumber:
        curstring += repr(result + current)
  except Exception as e:
    curstring = textnum
  return curstring

In [ ]:
# Sample of the above function
text2int(input_text)

'I see 5 cars 1 white 4 black 2 pedestrians '

I see 5 cars 1 white 4 black 2 pedestrians 

In [ ]:
import re
import json

def listToJson(keyword_item):
  dic = {}
  for keyword in keyword_item:
    #print(keyword)
    words = text2int(keyword)
    if re.match(r'[0-9] *', words):
      
      item_count = words.split(" ")
      if len(item_count) > 1:
        dic[item_count[1]] = item_count[0]
  return json.dumps(dic, indent=2)


**Below is the result for Approach 1**

In [ ]:
print(listToJson(keyword_items))

{
  "cars": "5",
  "black": "4",
  "pedestrians": "2"
}


```
{
  "cars": "5",
  "black": "4",
  "pedestrians": "2"
}
```

# **Approach 2. Using Ngrams and regular expressios**

In [ ]:
import nltk

In [ ]:
input_text = "I see five cars one white and four black and two pedestrians"

In [ ]:
allbigrams = list(nltk.ngrams(input_text.split(" "), 2))
allbigrams = [' '.join(t) for t in allbigrams]

In [ ]:
print(allbigrams)

['I see', 'see five', 'five cars', 'cars one', 'one white', 'white and', 'and four', 'four black', 'black and', 'and two', 'two pedestrians']


['I see', 'see five', 'five cars', 'cars one', 'one white', 'white and', 'and four', 'four black', 'black and', 'and two', 'two pedestrians']


**Result of Approach 2**

In [ ]:
print(listToJson(allbigrams))

{
  "cars": "5",
  "white": "1",
  "black": "4",
  "pedestrians": "2"
}


```
{
  "cars": "5",
  "white": "1",
  "black": "4",
  "pedestrians": "2"
}
```

# Other Approaches

**Solution 1**

a. Considering numbers & use chatbotNER
 
*   Converting words to number(Ex five to 5)
*   Finetune Chatbot NER to recognize the specific keywords and format result to Json
https://github.com/techthiyanes/chatbot_ner/blob/develop/ner_v2/detectors/numeral/number/en/data/units.csv

**Solution 2**

b. **Text to Text generation(T5)** - Encoder is text input and decoder is JSON output

*   Using T5(Text To Text Transfer Transformer) encoder and decorder architecture
*   Finetune the T5 model with our corpus of data with text as input and json as output


**Solution 3**

c. Use paraphrase then keybert(1,2/3 ngrams) then use semantic search then convert to JSON

*   Use T5 Paraphrase pass input text to model and generate text similar text with all necessary keywords present. 
*   Use Keybert with ngrams apporach and do the semantic search.
*   The semantic search results are next converted to Json output format.











